<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [12]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")


## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [13]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = 

with open('../../data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1175700
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "italy", "city": "serso", "street": "Via dei Caduti", "aqicn_url": "https://api.waqi.info/feed/A179560", "latitude": 46.07, "longitude": 11.25}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [14]:
# Retrieve feature groups
air_quality_roll_fg = fs.get_feature_group(
    name='air_quality',
    version=2,
)

weather_roll_fg = fs.get_feature_group(
    name='weather_roll',
    version=2,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


QUA BISOGNA CALCOLARE SUBITO IL PM25 ROLL WINDOW E POI SI MODIFICA LA RIGA QUA SOTTO CHE SI VA A MODIFICARE, QUINDI SI LEGGE PRIMA i dati fino ad ora e poi si calcola il pm25

In [15]:
df_aqi= air_quality_roll_fg.read()
df_aqi = df_aqi.sort_values(by='date', ascending=True)
df_aqi

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.52s) 


,date,pm25,pm25_3day_roll_mean,country,city,street,url
644,2020-11-10 00:00:00+00:00,5.030000,7.304875,italy,serso,Via dei Caduti,https://api.waqi.info/feed/A179560
318,2020-11-12 00:00:00+00:00,20.040001,7.304875,italy,serso,Via dei Caduti,https://api.waqi.info/feed/A179560
337,2020-11-13 00:00:00+00:00,20.299999,15.123333,italy,serso,Via dei Caduti,https://api.waqi.info/feed/A179560
1348,2020-11-14 00:00:00+00:00,21.990000,20.776667,italy,serso,Via dei Caduti,https://api.waqi.info/feed/A179560
1303,2020-11-15 00:00:00+00:00,19.230000,20.506666,italy,serso,Via dei Caduti,https://api.waqi.info/feed/A179560
...,...,...,...,...,...,...,...
884,2024-11-15 00:00:00+00:00,4.780000,4.780000,italy,serso,Via dei Caduti,https://api.waqi.info/feed/A179560
435,2024-11-16 00:00:00+00:00,4.090000,4.516667,italy,serso,Via dei Caduti,https://api.waqi.info/feed/A179560
250,2024-11-17 00:00:00+00:00,5.090000,4.653334,italy,serso,Via dei Caduti,https://api.waqi.info/feed/A179560
1330,2024-11-18 00:00:00+00:00,6.630000,5.270000,italy,serso,Via dei Caduti,https://api.waqi.info/feed/A179560


In [16]:
import requests
import pandas as pd

aq_today_df_roll = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
# Extract the current value (today_value) as a scalar
today_value = aq_today_df_roll["pm25"].iloc[0]  # Use .iloc[0] to get the scalar value
# Extract the last two values (past_2_d) as a list or scalars
past_2_d = df_aqi["pm25"].tail(2).tolist()  # Convert to a list for easier indexing
# Calculate the rolling average using scalar values
roll_aq = (today_value + past_2_d[0] + past_2_d[1]) / 3
#print(roll_aq)

aq_today_df_roll.insert(loc=1, column='pm25_3day_roll_mean', value=roll_aq)

aq_today_df_roll


,pm25,pm25_3day_roll_mean,country,city,street,date,url
0,19.0,24.21,italy,serso,Via dei Caduti,2024-11-20,https://api.waqi.info/feed/A179560


In [17]:
aq_today_df_roll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   pm25                 1 non-null      float32       
 1   pm25_3day_roll_mean  1 non-null      float64       
 2   country              1 non-null      object        
 3   city                 1 non-null      object        
 4   street               1 non-null      object        
 5   date                 1 non-null      datetime64[ns]
 6   url                  1 non-null      object        
dtypes: datetime64[ns](1), float32(1), float64(1), object(4)
memory usage: 180.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [18]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 45.75°N 11.5°E
Elevation 505.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-19,7.95,0.0,3.617955,185.710495,serso
1,2024-11-20,11.55,0.0,13.608762,322.523743,serso
2,2024-11-21,5.05,0.0,4.896530,143.972534,serso
3,2024-11-22,6.05,0.0,11.212135,317.602600,serso
4,2024-11-23,4.15,0.0,5.815978,158.198532,serso
5,2024-11-24,4.75,0.0,2.276840,161.564957,serso
6,2024-11-25,6.05,0.0,3.219938,153.435013,serso
7,2024-11-26,8.00,0.0,1.835647,191.309891,serso
8,2024-11-27,10.05,0.2,3.617955,174.289505,serso
9,2024-11-28,9.80,0.0,2.595997,146.309906,serso


In [19]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [20]:
# Insert new data
air_quality_roll_fg.insert(aq_today_df_roll)

2024-11-20 00:56:59,985 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1175700/fs/1166403/fg/1357144


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1175700/jobs/named/air_quality_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1b92e6d0f40>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25_3day_roll_mean",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 681996
         }
       },
       "result": {
         "observed_value": 24.210000038146973,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-19T11:56:59.000985Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_

In [21]:
df_aqi= air_quality_roll_fg.read()
df_aqi

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.24s) 


,date,pm25,pm25_3day_roll_mean,country,city,street,url
0,2021-05-31 00:00:00+00:00,2.06,1.986667,italy,serso,Via dei Caduti,https://api.waqi.info/feed/A179560
1,2022-09-06 00:00:00+00:00,6.72,4.896667,italy,serso,Via dei Caduti,https://api.waqi.info/feed/A179560
2,2023-06-17 00:00:00+00:00,5.07,4.123333,italy,serso,Via dei Caduti,https://api.waqi.info/feed/A179560
3,2022-01-08 00:00:00+00:00,8.49,5.290000,italy,serso,Via dei Caduti,https://api.waqi.info/feed/A179560
4,2021-04-06 00:00:00+00:00,0.85,2.120000,italy,serso,Via dei Caduti,https://api.waqi.info/feed/A179560
...,...,...,...,...,...,...,...
1393,2022-01-16 00:00:00+00:00,8.69,8.586667,italy,serso,Via dei Caduti,https://api.waqi.info/feed/A179560
1394,2020-11-18 00:00:00+00:00,5.92,10.993333,italy,serso,Via dei Caduti,https://api.waqi.info/feed/A179560
1395,2023-08-22 00:00:00+00:00,8.80,9.593333,italy,serso,Via dei Caduti,https://api.waqi.info/feed/A179560
1396,2021-02-08 00:00:00+00:00,8.52,15.173333,italy,serso,Via dei Caduti,https://api.waqi.info/feed/A179560


In [22]:
# Insert new data
weather_roll_fg.insert(daily_df)

2024-11-20 00:57:11,415 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1175700/fs/1166403/fg/1355121


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_roll_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1175700/jobs/named/weather_roll_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1b92e1c0490>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 678943
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-19T11:57:11.000415Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_betwe

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
